## Bibliotecas

In [1]:
import os
import re
import pickle
import numpy as np
from scipy.sparse import hstack, csr_matrix

## Importar objetos

In [2]:
# --> Modelo
with open("entrenamiento/modelo.pkl", "rb") as f:
    modelo = pickle.load(f)

In [3]:
# --> Vectorizador
with open("entrenamiento/vectorizador.pkl", "rb") as f:
    vectorizador = pickle.load(f)

## Funciones

In [4]:
def load_sel():
	"""
	Función que carga el diccionario de polaridades de SEL
	"""
	lexicon_sel = {}
	input_file = open('apoyo/SEL_full.txt', 'r')

	# Para cada línea del archivo
	for line in input_file:
		#Las líneas del lexicon tienen el siguiente formato:
		#abundancia	0	0	50	50	0.83	Alegría
		
		palabras = line.split("\t")					# Esto hara que se separe la linea en una lista de palabras
		palabras[6]= re.sub('\n', '', palabras[6])	# Esto hara que se elimine el salto de linea
		pair = (palabras[6], palabras[5])			# Esto hara que se guarde la palabra y su valor de polaridad

		if lexicon_sel:								# Si el diccionario no esta vacio
			if palabras[0] not in lexicon_sel:		# Si la palabra no esta en el diccionario
				lista = [pair]						# Se crea una lista con la palabra y su valor de polaridad
				lexicon_sel[palabras[0]] = lista	# Se agrega la lista al diccionario
			else:
				lexicon_sel[palabras[0]].append (pair)	# Si la palabra ya esta en el diccionario, se agrega la palabra y su valor de polaridad a la lista
		else:										# Si el diccionario esta vacio
			lista = [pair]							# Se crea una lista con la palabra y su valor de polaridad
			lexicon_sel[palabras[0]] = lista		# Se agrega la lista al diccionario

	input_file.close()								# Se cierra el archivo
	del lexicon_sel['Palabra']; #Esta llave se inserta porque es parte del encabezado del diccionario, por lo que se requiere eliminar
	#Estructura resultante
		#'hastiar': [('Enojo\n', '0.629'), ('Repulsi\xf3n\n', '0.596')]
	return lexicon_sel								# Se regresa el diccionario

def getSELFeatures(cadenas, lexicon_sel):
	"""
	Función que obtiene las características de SEL
	"""
	#'hastiar': [('Enojo\n', '0.629'), ('Repulsi\xf3n\n', '0.596')]
	features = []
	for cadena in cadenas:
		valor_alegria = 0.0
		valor_enojo = 0.0
		valor_miedo = 0.0
		valor_repulsion = 0.0
		valor_sorpresa = 0.0
		valor_tristeza = 0.0
		cadena_palabras = re.split('\s+', cadena)
		dic = {}
		for palabra in cadena_palabras:
			if palabra in lexicon_sel:
				caracteristicas = lexicon_sel[palabra]
				for emocion, valor in caracteristicas:
					if emocion == 'Alegría':
						valor_alegria = valor_alegria + float(valor)
					elif emocion == 'Tristeza':
						valor_tristeza = valor_tristeza + float(valor)
					elif emocion == 'Enojo':
						valor_enojo = valor_enojo + float(valor)
					elif emocion == 'Repulsión':
						valor_repulsion = valor_repulsion + float(valor)
					elif emocion == 'Miedo':
						valor_miedo = valor_miedo + float(valor)
					elif emocion == 'Sorpresa':
						valor_sorpresa = valor_sorpresa + float(valor)
		dic['__alegria__'] = valor_alegria
		dic['__tristeza__'] = valor_tristeza
		dic['__enojo__'] = valor_enojo
		dic['__repulsion__'] = valor_repulsion
		dic['__miedo__'] = valor_miedo
		dic['__sorpresa__'] = valor_sorpresa
		
		#Esto es para los valores acumulados del mapeo a positivo (alegría + sorpresa) y negativo (enojo + miedo + repulsión + tristeza)
		dic['acumuladopositivo'] = dic['__alegria__'] + dic['__sorpresa__']
		dic['acumuladonegative'] = dic['__enojo__'] + dic['__miedo__'] + dic['__repulsion__'] + dic['__tristeza__']
		
		features.append (dic)
	return features

In [5]:
def polaridad_cadenas(polaridades):
    """
    Esta función regresa un arreglo con los valores de polaridad de cada cadena
    """

    polaridad_cadenas = []
    for x in range(len(polaridades)):
        polaridad_cadena_pos = np.array([polaridades[x]['acumuladopositivo']])
        polaridad_cadena_neg = np.array([polaridades[x]['acumuladonegative']])
        polaridad_cadena = np.concatenate((polaridad_cadena_pos, polaridad_cadena_neg), axis=0)
        polaridad_cadenas.append(polaridad_cadena)
    return polaridad_cadenas

## Verificar si existe el archivo

In [6]:
if (os.path.exists('entrenamiento/lexicon_sel.pkl')):
		lexicon_sel_file = open ('entrenamiento/lexicon_sel.pkl','rb')
		lexicon_sel = pickle.load(lexicon_sel_file)
else:
	lexicon_sel = load_sel()
	lexicon_sel_file = open ('lexicon_sel.pkl','wb')
	pickle.dump(lexicon_sel, lexicon_sel_file)
	lexicon_sel_file.close()

## Predicción

In [7]:
texto_prueba = "Es un lugar muy poco agradable, no me gusta para nada"

In [9]:
# --> Procesar texto
texto_prueba = [texto_prueba]

vectorizador_prueba = vectorizador.transform(texto_prueba)

caracteristicas_prueba = getSELFeatures(texto_prueba, lexicon_sel)
polaridad_cadenas_prueba = polaridad_cadenas(caracteristicas_prueba)
matriz_prueba = csr_matrix(polaridad_cadenas_prueba)

vectorizador_prueba = hstack((vectorizador_prueba, matriz_prueba))


In [10]:
# --> Predicción
prediccion_prueba = modelo.predict(vectorizador_prueba)

# --> Resultados
print("Predicción: ", prediccion_prueba)

Predicción:  [5]


## Probar algoritmo

In [25]:
# --> Probabilidades
probabilidades_prueba = modelo.predict_proba(vectorizador_prueba)
print("Probabilidades: ", probabilidades_prueba)

Probabilidades:  [[0.0274188  0.01589131 0.11770293 0.15819847 0.68078849]]
